# Character Recognition Tool
### Created by 
### Joshua Baldwin - Michigan State University
### Ralf Schmaelzle - Michigan State University

# Instructions

## Install Packages

In [ ]:
#!pip install pliers
#!pip install python-magic
#!pip install face_recognition 
#!pip3 install --upgrade pandas
#!pip install libmagic
#!brew upgrade cmake
#!homebrew, ffmpeg, dlib
#!brew install  dlib
#!brew install libmagic

## Import Tools

In [ ]:
import os 
from os.path import join
import numpy as np

from pliers.tests.utils import get_test_data_path
from pliers.extractors import (FaceRecognitionFaceLocationsExtractor, FaceRecognitionFaceEncodingsExtractor, 
                               FaceRecognitionFaceLandmarksExtractor,SaliencyExtractor, BrightnessExtractor, merge_results)
from pliers.filters import FrameSamplingFilter
import face_recognition
from pliers.graph import Graph
import pandas as pd 
import sys


## Name your project and direct input files
### Please insert your project name
### Please direct your video files
### Please direct your reference images
### Please indicate the number of reference images
### Please indicate the names of the characters

In [ ]:
#Name the Project Below
project_name = 'Project_Name'

#Import Video
video_reference = 'Video_File'

#Import Reference Images 
images = os.listdir('Reference_Image_File')
images_reference = 'Reference_Image_File'
#How Many Known Images?
n_images = 7

#What are the Names of the Characters (Alphabetical)?
character_names = ['Character_1','Character_2','Character_nth']
n_characters = len(character_names)

## Set your frame rate

In [ ]:
rate = 1
sampler = FrameSamplingFilter(every = rate)
frames = sampler.transform(video_reference)
n_frames = frames.n_frames
n_frames_analyze = round(n_frames/24)
print('This video has ' + str(n_frames) + ' frames')

## Face Recognition Extractor

In [ ]:
character_images = []
for currs in character_names:
    cname = currs + '.jpg'
    character_images.append(cname)
    
character_images

In [ ]:
known_faces = []

for character in character_images:
    known_character_image =  face_recognition.load_image_file(images_reference + character)
    known_character_encoding = face_recognition.face_encodings(known_character_image)[0]
    #print (known_character_encoding)
    known_faces.append(known_character_encoding)
    

len(known_faces)

In [ ]:
res = []
for curr_frame_number in np.arange(0, n_frames, 1):
    sys.stdout.write(" %d, \r" % (curr_frame_number) )
    sys.stdout.flush()
    curr_frame = frames.get_frame(curr_frame_number).video.get_frame(curr_frame_number).video.get_frame(curr_frame_number).data
    face_locations = face_recognition.face_locations(curr_frame)
    face_encodings = face_recognition.face_encodings(curr_frame, face_locations)

    if len(face_encodings)>0 :
        results = face_recognition.compare_faces(known_faces, face_encodings[0])
        res.append(results)
    else:
        res.append([False] * n_images)
        
res2 = np.asarray(res)
np.asarray(res2[0])

new_result = np.zeros((n_frames,n_characters))

for this_line in range(n_frames):
    new_result[this_line,:] = np.asarray(res2[this_line])

df2 = pd.DataFrame( data    = new_result,
                    columns = character_names );
df2.to_csv('../data/output_data/' + project_name + '_face_recognition.csv')
df2